In [26]:
!pip install pybullet

In [27]:
import pybullet as p
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# Prepare Data

In [28]:
# Trajectory Data
TRAJECTORY_DATA_FILENAME = 'trajectory_data.csv'
# End Effector Positions
END_EFFECTOR_POSITIONS_DATA = 'end_effector_positions_data.csv'

In [29]:
# Load trajectory data
end_effector_df = pd.read_csv(END_EFFECTOR_POSITIONS_DATA)
end_effector_df

,PairID,Algorithm,X,Y,Z
0,1,RRT,1.554010,-0.045191,0.728940
1,1,RRT,1.553890,-0.045681,0.729443
2,1,RRT,1.553550,-0.047151,0.730952
3,1,RRT,1.552950,-0.049600,0.733469
4,1,RRT,1.552060,-0.053021,0.736999
...,...,...,...,...,...
743775,1993,RRT*,0.309760,-0.069059,0.435788
743776,1993,RRT*,0.312857,-0.063055,0.436939
743777,1993,RRT*,0.315044,-0.058943,0.437726
743778,1993,RRT*,0.316268,-0.056684,0.438158


In [30]:
# Load end-effector data
traj_df = pd.read_csv(TRAJECTORY_DATA_FILENAME)
traj_df

,PairID,Algorithm,Time,Joint1_Position,Joint1_Velocity,Joint1_Acceleration,Joint2_Position,Joint2_Velocity,Joint2_Acceleration,Joint3_Position,...,Joint4_Acceleration,Joint5_Position,Joint5_Velocity,Joint5_Acceleration,Joint6_Position,Joint6_Velocity,Joint6_Acceleration,Joint7_Position,Joint7_Velocity,Joint7_Acceleration
0,1,RRT,0.0000,0.321656,0.527207,2.56334,1.51281,0.300186,1.94992,-2.834730,...,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,1,RRT,0.1000,0.321485,0.526847,2.56209,1.51230,0.300083,1.94907,-2.834730,...,-2.065800e-03,-1.697270e-02,2.637100e-05,-0.034214,-0.072007,-0.250000,-0.10221,-0.020658,-0.169727,0.000264
2,1,RRT,0.2000,0.320971,0.525767,2.55834,1.51077,0.299773,1.94652,-2.834730,...,-4.131610e-03,-3.394540e-02,5.274190e-05,-0.034214,-0.072007,-0.250000,-0.10221,-0.020658,-0.169727,0.000264
3,1,RRT,0.3000,0.320116,0.523967,2.55209,1.50822,0.299257,1.94228,-2.834720,...,-6.197410e-03,-5.091810e-02,7.911290e-05,-0.034214,-0.072007,-0.250000,-0.10221,-0.020658,-0.169727,0.000264
4,1,RRT,0.4000,0.318919,0.521447,2.54334,1.50464,0.298534,1.93634,-2.834710,...,-8.263220e-03,-6.789080e-02,1.054840e-04,-0.034214,-0.072007,-0.250000,-0.10221,-0.020658,-0.169727,0.000264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743775,1993,RRT*,16.5000,-2.519190,1.861790,1.29873,1.11843,1.930630,1.45083,0.442102,...,1.167860e-01,2.469750e-02,9.352940e-02,0.081184,-0.036330,0.029577,-0.31250,-0.312844,-0.066159,-0.250545
743776,1993,RRT*,16.6000,-2.521820,1.862970,1.29777,1.12853,1.940740,1.45297,0.450202,...,8.550140e-02,1.808160e-02,6.847490e-02,0.081184,-0.036330,0.029577,-0.31250,-0.312844,-0.066159,-0.250545
743777,1993,RRT*,16.7000,-2.523630,1.863780,1.29711,1.13551,1.947730,1.45445,0.455797,...,5.421700e-02,1.146570e-02,4.342040e-02,0.081184,-0.036330,0.029577,-0.31250,-0.312844,-0.066159,-0.250545
743778,1993,RRT*,16.8000,-2.524630,1.864230,1.29674,1.13937,1.951590,1.45526,0.458886,...,2.293260e-02,4.849730e-03,1.836590e-02,0.081184,-0.036330,0.029577,-0.31250,-0.312844,-0.066159,-0.250545


In [31]:
# BUG!
# Select the joint positions
joint_positions = traj_df.iloc[:, [3, 4, 5, 6, 7, 8, 9]].to_numpy()

# Select the XYZ coordinates of the end-effector
end_effector_positions = end_effector_df[['X', 'Y', 'Z']].to_numpy()

In [32]:
# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(joint_positions, end_effector_positions, test_size=0.2, random_state=42)

# Model

Use GPU if available

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


Create a PyTorch Dataset

In [34]:
class TrajectoryDataset(Dataset):
    def __init__(self, joint_data, position_data):
        self.joint_data = torch.tensor(joint_data, dtype=torch.float32)
        self.position_data = torch.tensor(position_data, dtype=torch.float32)

    def __len__(self):
        return len(self.joint_data)

    def __getitem__(self, idx):
        return self.joint_data[idx], self.position_data[idx]

# Create dataset objects
train_dataset = TrajectoryDataset(X_train, y_train)
test_dataset = TrajectoryDataset(X_test, y_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

Build the Neural Network

In [36]:
class PositionModel(nn.Module):
    def __init__(self):
        super(PositionModel, self).__init__()
        self.fc1 = nn.Linear(7, 128)  # 7 joint positions input
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 3)  # 3 coordinates (X, Y, Z) output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = PositionModel().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

# Set the number of training epochs
epochs = 50

# Call the train_model function
train_model(model, train_loader, criterion, optimizer, epochs)

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    print(f'Test Loss: {total_loss / len(test_loader)}')

evaluate_model(model, test_loader, criterion)

Epoch 1/50, Loss: 0.0007776137790642679
Epoch 2/50, Loss: 0.0012338111409917474
Epoch 3/50, Loss: 0.0005335173918865621
Epoch 4/50, Loss: 0.0003185989917255938
Epoch 5/50, Loss: 0.00031868237419985235
Epoch 6/50, Loss: 0.00035667282645590603
Epoch 7/50, Loss: 0.0006241612136363983
Epoch 8/50, Loss: 0.0004481697687879205
Epoch 9/50, Loss: 0.0003747241571545601
Epoch 10/50, Loss: 0.0003650840080808848
Epoch 11/50, Loss: 0.0005013883928768337
Epoch 12/50, Loss: 0.00035275518894195557
Epoch 13/50, Loss: 0.0005746633396483958
Epoch 14/50, Loss: 0.0005276569281704724
Epoch 15/50, Loss: 0.0005040646065026522
Epoch 16/50, Loss: 0.00028823173488490283
Epoch 17/50, Loss: 0.00014774987357668579
Epoch 18/50, Loss: 0.0002670660032890737
Epoch 19/50, Loss: 0.00020025142293889076
Epoch 20/50, Loss: 0.00027692425646819174
Epoch 21/50, Loss: 0.00031478251912631094
Epoch 22/50, Loss: 0.00024409509205725044
Epoch 23/50, Loss: 0.0003422281879466027
Epoch 24/50, Loss: 0.00025571996229700744
Epoch 25/50, Lo

In [37]:
torch.save(model.state_dict(), 'traj2position_model.pth')

Inference

In [41]:
# Create model architecture
model = PositionModel().to(device)

# Load the model weights
model.load_state_dict(torch.load('traj2position_model.pth'))

# Switch to inference mode
model.eval()

PositionModel(
  (fc1): Linear(in_features=7, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
)

In [49]:
# Sample input joint positions
sample_joint_positions = torch.tensor([[-0.457362,1.32912,-0.540427,0.467519,0.665829,-0.940394,0.166331]], device=device)

# Examples
# -0.457362,1.32912,-0.540427,0.467519,0.665829,-0.940394,0.166331 -> 1.68442,-0.233629,0.8497
# 0.192307,1.39618,1.82676,-0.621988,-1.6764,-1.28055,-0.822372 -> 1.66657,0.359841,0.701665
# 0.188742,1.3806,1.77671,-0.621967,1.51648,1.29734,2.31899 -> 1.66651,0.359853,0.70174

# Perform inference
with torch.no_grad():
    predicted_position = model(sample_joint_positions)

print(predicted_position.cpu().numpy())

# 1.68442,-0.233629,0.8497

[[ 1.6778855  -0.23101458  0.8409976 ]]


# Pybullet Traj2Position

In [25]:
p.connect(p.DIRECT)
robot_id = p.loadURDF("iiwa14_standalone.urdf")

# Set the joint angles (this example assumes 7 joints for iiwa)
joint_angles = [2.80694,-1.89474,-0.355364,-1.5175,-2.40752,-0.513802,-0.448375]  # Your joint angles
for i, angle in enumerate(joint_angles):
    p.resetJointState(robot_id, i, angle)

# Compute forward kinematics
link_state = p.getLinkState(robot_id, 8)
position, orientation = link_state[0], link_state[1]
position #1.55404,-0.0452534,0.728915

(-0.36395739807334543, 0.10161573604522772, 0.4425315222181057)